In [1]:
import pandas as pd
import numpy as np
from random import randint
import math
from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv("cc_clustering_data.csv")
credit_df = pd.DataFrame(data)
credit_df = credit_df.set_index('CUST_ID')
credit_df = credit_df.dropna()
norm_df=(credit_df-credit_df.min())/(credit_df.max()-credit_df.min())

In [3]:
def find_distance(curr_df, row1_idx, row2_idx):
    row1 = curr_df.iloc[row1_idx]
    row2 = curr_df.iloc[row2_idx]
    dist = np.sqrt(np.sum([ (a-b)*(a-b) for a, b in zip(row1, row2)]))
    return dist

In [4]:
def count_bits(num):
    res = 0
    while num > 0:
        num = num & (num - 1)
        res = res + 1
    return res

In [5]:
def k_center_greedy(local_df, num_k):
    centers_list = []
    num_rows = local_df.shape[0]
    centers_list.append(randint(0, num_rows - 1))
    for i in tqdm(range(num_k - 1), leave = False):
        max_dist, max_dist_pt = 0, centers_list[0]
        for row_id in range(num_rows):
            row_dist = min([find_distance(local_df, row_id, j) for j in centers_list])
            if (row_dist > max_dist):
                max_dist = row_dist
                max_dist_pt = row_id
        centers_list.append(max_dist_pt)
    cost_clustering = max(min([find_distance(local_df, row_id, j) for j in centers_list]) for row_id in range(num_rows))
    return cost_clustering

In [18]:
def generate_bitmasks_list(k , n):
    res = []
    bitmask = (1 << k) - 1
    while bitmask <= ( (1 << n) -(1 << (n-k)) ):
        res.append(bitmask)
        a = bitmask & (-bitmask)
        b = bitmask + a
        bitmask = b | (((b ^ bitmask) >> 2)//a)
    return res

In [7]:
def k_center_optimal(local_df, num_k):
    clustering_cost = math.inf
    num_rows = local_df.shape[0]
    allowed_bitmasks = generate_bitmasks_list(num_k, num_rows)
    for bitmask in tqdm(allowed_bitmasks, leave = False):
        centers_list = []
        for row_id in range(num_rows):
            if (bitmask & (1 << row_id)):
                centers_list.append(row_id)
        clustering_cost_candidate = max(min([find_distance(local_df, r_id, j) for j in centers_list]) for r_id in range(num_rows))
        clustering_cost = min(clustering_cost, clustering_cost_candidate)
    return clustering_cost
        

### Greedy Algo over all DataSet

In [20]:
for k in [2, 4, 10]:
    print("For Number of Clusters: ", k)
    greedy_objective_value = k_center_greedy(norm_df, k)
    print("Objective value of Greedy Algo :", greedy_objective_value, "\n")

For Number of Clusters:  2


  0%|          | 0/1 [00:00<?, ?it/s]

Objective value of Greedy Algo : 2.1593968934215253 

For Number of Clusters:  4


  0%|          | 0/3 [00:00<?, ?it/s]

Objective value of Greedy Algo : 1.846269438999143 

For Number of Clusters:  10


  0%|          | 0/9 [00:00<?, ?it/s]

Objective value of Greedy Algo : 1.4913641238206745 



### Greedy v/s Optimal on Sample size of 20 

In [19]:
sample_df = norm_df.sample(n = 20)
for k in [2, 4, 10]:
    print("For Number of Clusters: ", k)
    greedy_objective_value = k_center_greedy(sample_df, k)
    optimal_objective_value = k_center_optimal(sample_df, k)
    approx_factor = greedy_objective_value / optimal_objective_value
    print("Objective value of Greedy Algo :", greedy_objective_value)
    print("Objective Value of Optimal Algo:", optimal_objective_value)
    print("Approximation Factor :", approx_factor, "\n\n")

For Number of Clusters:  2


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

Objective value of Greedy Algo : 1.464181001245336
Objective Value of Optimal Algo: 1.0535302518820355
Approximation Factor : 1.3897854367539142 


For Number of Clusters:  4


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4845 [00:00<?, ?it/s]

Objective value of Greedy Algo : 1.1018027406242379
Objective Value of Optimal Algo: 0.9636419999491286
Approximation Factor : 1.1433735149385382 


For Number of Clusters:  10


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/184756 [00:00<?, ?it/s]

Objective value of Greedy Algo : 0.511764351630979
Objective Value of Optimal Algo: 0.4616924620962943
Approximation Factor : 1.1084529067408542 


